In [1]:
import numpy as np 
import cvxopt
import cvxopt.solvers

In [2]:
variance = 0.1
n = int(500/2**2)
x_00 = np.hstack((np.random.normal(0,variance,(n,1)) , np.random.normal(0,variance,(n,1)) ))
x_01 = np.hstack((np.random.normal(0,variance,(n,1)) , np.random.normal(1,variance,(n,1)) ))
x_10 = np.hstack((np.random.normal(1,variance,(n,1)) , np.random.normal(0,variance,(n,1)) ))
x_11 = np.hstack((np.random.normal(1,variance,(n,1)) , np.random.normal(1,variance,(n,1)) ))
x = np.asarray(np.concatenate((x_00,x_01,x_10,x_11)))
print(x.shape)
# y_train = np.asmatrix(np.append(np.zeros((3*n,1)),np.ones((n,1)))).T
y = np.asarray(np.concatenate((-1 * np.ones((n,1)),np.ones((2*n,1)),(-1 * np.ones((n,1)))))).reshape(-1)

### Trying to solve classification problem , so commenting the below noise
# y_train_and += np.random.normal(1,variance,(4*n,1))
# y_train_or += np.random.normal(1,variance,(4*n,1))
# y_train_xor += np.random.normal(1,variance,(4*n,1))
print(y)

(500, 2)
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  

In [40]:
def rbf_kernel(x,y, gamma = 0.1):
        return np.exp(-1.0*gamma*np.dot(np.subtract(x,y).T,np.subtract(x,y)))

def linear(x,y):
    return np.dot(x.T,y)

def apply_kernel(X, gamma = 0.1):
    K = np.zeros([X.shape[0],X.shape[0]])
    for i in range(X.shape[0]):
        for j in range(X.shape[0]):
            if(kernel == 'rbf'):
                K[i,j] = rbf_kernel(X[i],X[j], gamma)
            else:
                K[i,j] = linear(X[i],X[j])
    return K

In [41]:
num_data, num_features = x.shape
labels = y.astype(np.double)
K = apply_kernel(x)
P = cvxopt.matrix(np.outer(labels,labels)*K)
q = cvxopt.matrix(np.ones(num_data)*-1)
A = cvxopt.matrix(labels,(1,num_data))
b = cvxopt.matrix(0.0)
G = cvxopt.matrix(np.diag(np.ones(num_data) * -1))
h = cvxopt.matrix(np.zeros(num_data))

alphas = np.ravel(cvxopt.solvers.qp(P, q, G, h, A, b)['x'])
is_sv = alphas>1e-5

     pcost       dcost       gap    pres   dres
 0: -1.9233e+02 -5.1043e+02  3e+02  2e-15  2e+00
 1: -2.4938e+02 -3.7131e+02  1e+02  1e-13  7e-01
 2: -3.1664e+02 -4.4297e+02  1e+02  1e-13  7e-01
 3: -3.8058e+02 -5.1286e+02  1e+02  7e-14  7e-01
 4: -6.4998e+02 -8.2530e+02  2e+02  7e-14  7e-01
 5: -1.1139e+03 -1.4050e+03  3e+02  1e-13  4e-01
 6: -1.2983e+03 -1.3900e+03  9e+01  2e-13  7e-02
 7: -1.3076e+03 -1.3190e+03  1e+01  4e-13  6e-03
 8: -1.3044e+03 -1.3124e+03  8e+00  2e-13  1e-03
 9: -1.3106e+03 -1.3108e+03  2e-01  1e-13  3e-05
10: -1.3107e+03 -1.3107e+03  3e-03  7e-13  3e-07
11: -1.3107e+03 -1.3107e+03  3e-05  3e-13  3e-09
Optimal solution found.


In [42]:
support_vectors = x[is_sv]
n_support = np.sum(is_sv)
alphas = alphas[is_sv]
support_labels = labels[is_sv]
indices = np.arange(num_data)[is_sv]
intercept = 0
for i in range(alphas.shape[0]):
    intercept += support_labels[i] 
    intercept -= np.sum(alphas*support_labels*K[indices[i],is_sv])
intercept /= alphas.shape[0]


In [44]:
variance = 0.1
n = int(500/2**2)
x_00 = np.hstack((np.random.normal(0,variance,(n,1)) , np.random.normal(0,variance,(n,1)) ))
x_01 = np.hstack((np.random.normal(0,variance,(n,1)) , np.random.normal(1,variance,(n,1)) ))
x_10 = np.hstack((np.random.normal(1,variance,(n,1)) , np.random.normal(0,variance,(n,1)) ))
x_11 = np.hstack((np.random.normal(1,variance,(n,1)) , np.random.normal(1,variance,(n,1)) ))
x_test = np.asarray(np.concatenate((x_00,x_01,x_10,x_11)))
print(x.shape)
# y_train = np.asmatrix(np.append(np.zeros((3*n,1)),np.ones((n,1)))).T
y_test = np.asarray(np.concatenate((-1 * np.ones((n,1)),np.ones((2*n,1)),(-1 * np.ones((n,1)))))).reshape(-1)

### Trying to solve classification problem , so commenting the below noise
print(y_test.shape)

(500, 2)
(500,)


In [47]:
score = np.zeros(x_test.shape[0])
for i in range(x_test.shape[0]):
    s = 0
    for alpha,label,sv in zip(alphas,support_labels,support_vectors):
        s += alpha*label*rbf_kernel(x_test[i],sv)
    score[i] = s
score = score + intercept

res = np.where(score>0,1,-1)
print(res)

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1

In [46]:
print(res == y_test)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T